In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
from neural_network.trainer import Agent
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib

In [4]:
agent = Agent('resnet50');

Setup configurations...
Dataset sizes - Training: 421 Validation: 47 Test: 0
Loading architecture from logs/tb_logs/lightning/resnet50/version_2/checkpoints/epoch=34-step=2424.ckpt (checkpoint)..
Model [LightningModel] was created
logs/tb_logs/


GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [5]:
agent.fit()

initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 46.2 M
---------------------------------
46.2 M    Trainable params
0         Non-trainable params
46.2 M    Total params
184.645   Total estimated model params size (MB)



Epoch 1 [71/71] {'loss': '0.482'}}
Epoch 2 [71/71] {'loss': '0.315'}
Epoch 3 [71/71] {'loss': '0.0975'}
Epoch 4 [71/71] {'loss': '0.0263'}
Epoch 5 [71/71] {'loss': '0.105'}}
Epoch 6 [71/71] {'loss': '0.0547'}
Epoch 7 [71/71] {'loss': '0.0128'}}
Epoch 8 [71/71] {'loss': '0.00575'}
Epoch 9 [71/71] {'loss': '0.0176'}}
Epoch 10 [71/71] {'loss': '0.128'}}
Epoch 11 [71/71] {'loss': '0.106'}
Epoch 12 [71/71] {'loss': '0.115'}}
Epoch 13 [71/71] {'loss': '0.357'}}
Epoch 14 [71/71] {'loss': '0.242'}
Epoch 15 [71/71] {'loss': '0.0699'}
Epoch 16 [71/71] {'loss': '0.102'}}
Epoch 17 [71/71] {'loss': '0.0161'}
Epoch 18 [71/71] {'loss': '0.0215'}}
Epoch 19 [71/71] {'loss': '0.00295'}
Epoch 20 [71/71] {'loss': '0.00482'}}
Epoch 21 [71/71] {'loss': '0.00422'}
Epoch 22 [71/71] {'loss': '0.00154'}}
Epoch 23 [71/71] {'loss': '0.000957'}
Epoch 24 [71/71] {'loss': '0.00247'}}
Epoch 25 [71/71] {'loss': '0.00187'}}
Epoch 26 [71/71] {'loss': '0.000552'}
Epoch 27 [71/71] {'loss': '0.000467'}
Epoch 28 [71/71] {'

/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  4853.4         	|  100 %          	|
------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  65.465         	|74             	|  4844.4         	|  99.815         	|
run_training_batch                 	|  0.73843        	|5196           	|  3836.9         	|  79.056         	|
optimizer_step_and_closure_0    

# Setup Input!

In [6]:
#agent.load_model()

# Example model to plot CAMS

In [ ]:
from neural_network.utils.cam import get_cam
from neural_network.utils.cam_plots import plot_CAM_grid
from neural_network.utils import move_to_device, to_cpu_numpy
from neural_network.utils import interactive_slices, interactive_slices_masked
def cam_example(agent, extractor_name:str='SmoothGradCAMpp',img_size:tuple=(79,224,224), plot_type:str='grid',cmap:str='jet', alpha:float=0.3, observed_class:str=None, load_image:str='CN'):
    #label = 0
    target_layer = 'model.layer4'
    model = type(agent.model.model).__name__
    
    image = {
        'CN':nib.load('data/SPM_categorised/AIH/CN/CN_ADNI_998.nii').get_fdata,
        'MCI':nib.load('data/SPM_categorised/AIH/MCI/MCI_ADNI_1586.nii').get_fdata,
        'AD':nib.load('data/SPM_categorised/AIH/AD/AD_ADNI_2975.nii').get_fdata
    }[load_image]()
    label_to_class = {
        0:'CN',
        1:'MCI',
        2:'AD'
    }
    class_to_label = {v: k for k, v in label_to_class.items()}
    
    image = torch.from_numpy(resize(image, img_size)).float()
    model = agent.model

    model, image = move_to_device(model, image, 'cuda')
    
    mask, predicted_label = get_cam(model, image, extractor_name=extractor_name, target_layer=target_layer, observed_class=class_to_label[observed_class])
    
    #print("Number of slices above threshold:", sum(mask > 150))
    
    
    observed_class = observed_class if observed_class else load_image
    
    if plot_type == 'grid':
        fig = plot_CAM_grid(to_cpu_numpy(image), mask,layer=target_layer, label=label_to_class[predicted_label], observed_class=observed_class,extractor=extractor_name,cmap=cmap, alpha=alpha)
        
    elif plot_type == 'slice':
        testplot = interactive_slices()
        testplot.multi_slice_viewer(to_cpu_numpy(image))
        #testplot.cycle(0.1)
        testplot.close()
    elif plot_type == 'slice_masked':
        testplot = interactive_slices_masked()
        testplot.multi_slice_viewer(to_cpu_numpy(image), mask)
        #testplot.cycle(1)
        
        testplot.close()
    return (fig,mask, predicted_label)
        

In [13]:
# Iterate all cams
import time
from torch.utils.tensorboard import SummaryWriter
tb_writer = SummaryWriter(log_dir=f'logs/visualisation/version_{round(time.time())}', filename_suffix='.CAM')

plot_cams = {
    'CAM':{},
    'GradCAM':{},
    'GradCAMpp':{},
    'SmoothGradCAMpp':{},
    'Saliency':{'cmap':'hot', 'alpha':1},
    'ScoreCAM':{},
    'SSCAM':{},
    'ISCAM':{},
}
images = ['CN','MCI','AD']#[:1]
observed_classes = ['CN','MCI','AD']
errors = []

for extractor_name, params in plot_cams.items():
    for image in images:
        for observed_class in observed_classes:
            try:
                fig,_,_ = cam_example(agent,extractor_name=extractor_name, **params,load_image=image,observed_class=observed_class)
                tb_writer.add_figure(f"{extractor_name}/{image}/{observed_class}",fig)

            except RuntimeError as e:
                errors.append((f'Model: {extractor_name} {e}'))

In [9]:
%matplotlib inline

In [10]:
plot_cams = {
    'CAM':{},
    'GradCAM':{},
    'GradCAMpp':{},
    'SmoothGradCAMpp':{},
    'Saliency':{'cmap':'hot', 'alpha':1},
    'ScoreCAM':{},
    'SSCAM':{},
    'ISCAM':{},
}
images = ['CN','MCI','AD']#[:1]
observed_classes = ['CN','MCI','AD']
fig,mask,_ = cam_example(agent,extractor_name='SmoothGradCAMpp', **params,load_image='AD',observed_class='CN')
mask 

array([[[1.06413430e-03, 3.18989344e-03, 2.24037934e-03, 4.07983243e-05,
         1.64672697e-03, 3.40794376e-03, 3.04511283e-03],
        [2.05302909e-02, 8.99572521e-02, 1.09509155e-01, 9.21289399e-02,
         9.66381952e-02, 4.49617058e-02, 3.85731272e-02],
        [3.77208740e-02, 2.85771847e-01, 3.13196659e-01, 1.88670173e-01,
         1.72440961e-01, 5.05159199e-02, 3.75668444e-02],
        [5.57416864e-02, 3.88459951e-01, 4.27888453e-01, 2.09720522e-01,
         1.27726614e-01, 6.14761077e-02, 4.40539792e-02],
        [5.13880998e-02, 3.73264939e-01, 3.93005818e-01, 1.86727062e-01,
         8.91030133e-02, 7.03534037e-02, 3.61692198e-02],
        [3.57444398e-02, 2.23318651e-01, 2.03232110e-01, 9.55260918e-02,
         7.71973133e-02, 7.11510852e-02, 3.74832042e-02],
        [1.34040527e-02, 6.65361062e-02, 4.81477119e-02, 1.41306072e-02,
         2.75531486e-02, 2.22721286e-02, 3.03215720e-02]],

       [[3.56701110e-03, 8.15110689e-04, 0.00000000e+00, 2.49054725e-03,
        

In [11]:
np.sum(mask*255 > 150)

19

In [12]:
sum(np_mask[0]>150)

NameError: name 'np_mask' is not defined

In [ ]:
np_mask = to_np_figure(mask).numpy()

In [ ]:
retri.shape

In [ ]:
np_mask[to_np_figure(mask)>150].shape

In [ ]:
plt.imshow(to_np_fig_grid)

In [ ]:
from neural_network.utils.utils import to_np_figure, to_np_fig_grid

len(sum(numpy_to_image_grid(mask)>160))

In [ ]:
plt.imshow((fig*255).astype(np.int8))

In [ ]:
%matplotlib widget
cam_example(agent,extractor_name='GradCAM', plot_type='slice_masked')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure()
plt.imshow((first_image[0].squeeze(0)[50] * 255).numpy().astype(np.uint8), cmap='gray')
plt.imshow((darp[50] * 255).astype(np.uint8), cmap='jet', alpha=0.3)

plt.show()

In [ ]:
from neural_network.utils.utils import interactive_slices
testplot = interactive_slices()
testplot.multi_slice_viewer(darp)

In [ ]:
from neural_network.cam.cam import SmoothGradCAMpp

In [ ]:
wrapped_model = SmoothGradCAMpp(model,target_layer=model.model.layer4[1].conv2)

In [ ]:
first_image[0].unsqueeze(0).shape

In [ ]:
model.eval()
cam, idx = wrapped_model(first_image[0].unsqueeze(0))